In [4]:
import os
import sys

import pandas as pd
from pandas_datareader import wb

import warnings

# %load_ext watermark

%watermark -v -p os,sys,pandas,pandas_datareader

Python implementation: CPython
Python version       : 3.10.8
IPython version      : 8.8.0

os               : unknown
sys              : 3.10.8 (main, Nov 24 2022, 08:08:27) [Clang 14.0.6 ]
pandas           : 1.5.2
pandas_datareader: 0.10.0



Python implementation: CPython
Python version       : 3.10.8
IPython version      : 8.8.0

os               : unknown
sys              : 3.10.8 (main, Nov 24 2022, 08:08:27) [Clang 14.0.6 ]
pandas           : 1.5.2
pandas_datareader: 0.10.0


In [2]:
warnings.filterwarnings("ignore")
#warnings.filterwarnings("defalt")

In [3]:
# Global Variables

data_file_path = 'Data'
image_file_path = 'Image'


In [4]:
wb_countries_df = wb.get_countries()

countries_only_list = wb_countries_df[wb_countries_df['region']!='Aggregates']['name'].reset_index(drop=True)


In [5]:
migration = wb.WorldBankReader(symbols=['SM.POP.NETM'], start=2000, end=2019, countries='all').read().reset_index()
population = wb.WorldBankReader(symbols=['SP.POP.TOTL'], start=2000, end=2019, countries='all').read().reset_index()


In [42]:
with open(os.path.join ( data_file_path, 'data_stats.txt' ) , 'w') as text_file:
    sys.stdout = text_file

    print("migration   : ", migration.columns.tolist())
    print("")
    print("migration.info() : ")
    print("")
    print(migration.info())
    print("")
    print("migration.describe() : ")
    print("")
    print(migration.describe())
    print("")

# Reset stdout to its original value
sys.stdout = sys.__stdout__

In [43]:
with open(os.path.join ( data_file_path, 'data_stats.txt' ) , 'a') as text_file:
    sys.stdout = text_file

    print("population   : ", population.columns.tolist())
    print("")
    print("population.info() : ")
    print("")
    print(population.info())
    print("")
    print("population.describe() : ")
    print("")
    print(population.describe())

# Reset stdout to its original value
sys.stdout = sys.__stdout__

In [16]:
migration_df = pd.merge(population, migration, on=['country', 'year'], how='outer')

migration_df = migration_df[migration_df['country'].isin(countries_only_list)] # Trim to only countries

migration_df.rename(columns={'SP.POP.TOTL': 'population', 'SM.POP.NETM': 'migration_count'}, inplace = True)

migration_df['year'] = migration_df['year'].astype(int) # Make int so you can sort

migration_df.sort_values(by = ['country', 'year'], inplace = True) # sort
migration_df.reset_index(drop = True, inplace = True) # Reindex dropping old index

print(migration_df)

          country  year  population  migration_count
0     Afghanistan  2000  19542982.0       -1007135.0
1     Afghanistan  2001  19688632.0        -192286.0
2     Afghanistan  2002  21000256.0        1327074.0
3     Afghanistan  2003  22645130.0         388632.0
4     Afghanistan  2004  23553551.0        -248616.0
...           ...   ...         ...              ...
4335     Zimbabwe  2015  14154937.0         -59918.0
4336     Zimbabwe  2016  14452704.0         -59918.0
4337     Zimbabwe  2017  14751101.0         -59918.0
4338     Zimbabwe  2018  15052184.0         -59918.0
4339     Zimbabwe  2019  15354608.0         -59918.0

[4340 rows x 4 columns]


In [18]:
# Add calculated features to migration_df

migration_df['migration_count_change'] = migration_df.groupby('country')['migration_count'].diff()

migration_df['migration_percent'] = migration_df['migration_count'] / migration_df['population'] * 100

migration_df['migration_percent_change'] = migration_df.groupby('country')['migration_percent'].diff()

print("migration_df   : ", migration_df.columns.tolist())
print(migration_df)


migration_df   :  ['country', 'year', 'population', 'migration_count', 'migration_count_change', 'migration_percent', 'migration_percent_change']
          country  year  population  migration_count  migration_count_change  \
0     Afghanistan  2000  19542982.0       -1007135.0                     NaN   
1     Afghanistan  2001  19688632.0        -192286.0                814849.0   
2     Afghanistan  2002  21000256.0        1327074.0               1519360.0   
3     Afghanistan  2003  22645130.0         388632.0               -938442.0   
4     Afghanistan  2004  23553551.0        -248616.0               -637248.0   
...           ...   ...         ...              ...                     ...   
4335     Zimbabwe  2015  14154937.0         -59918.0                     0.0   
4336     Zimbabwe  2016  14452704.0         -59918.0                     0.0   
4337     Zimbabwe  2017  14751101.0         -59918.0                     0.0   
4338     Zimbabwe  2018  15052184.0         -59918.0  

In [19]:
# Save to Master csv file
print("migration_df   : ", migration_df.columns.tolist())

migration_df.to_csv ( os.path.join ( data_file_path, "Master_Global_Data_2000_2019.csv" ), index=False  )

migration_df   :  ['country', 'year', 'population', 'migration_count', 'migration_count_change', 'migration_percent', 'migration_percent_change']


In [21]:
# Calculate global annual totals and consistency check net_flow ~ zero

global_data = pd.DataFrame({'year': range(2000, 2020)})

global_data['population'] = migration_df.groupby('year')['population'].sum().reset_index()['population']

# Filter the migration data to include only rows with negative migration counts
migration_outflows_df = migration_df[migration_df['migration_count'] < 0]
migration_inflows_df  = migration_df[migration_df['migration_count'] > 0]

# Group by year and calculate the sum of migration counts
migration_outflows_total = migration_outflows_df.groupby('year')['migration_count'].sum().reset_index()
migration_inflows_total = migration_inflows_df.groupby('year')['migration_count'].sum().reset_index()

global_data['migration_outflows'] = migration_outflows_total['migration_count']
global_data['migration_inflows'] = migration_inflows_total['migration_count']
global_data['migration_flow_percent'] = global_data['migration_inflows'] / global_data['population'] * 100

global_data['migration_net'] = migration_df.groupby('year')['migration_count'].sum().reset_index()['migration_count']

global_data['migration_net_percent']  = global_data['migration_net'] / global_data['population']

print("global_data   : ", global_data.columns.tolist())

print(global_data.head(25))



global_data   :  ['year', 'population', 'migration_outflows', 'migration_inflows', 'migration_flow_percent', 'migration_net', 'migration_net_percent']
    year    population  migration_outflows  migration_inflows  \
0   2000  6.122149e+09          -6080517.0          6094612.0   
1   2001  6.204009e+09          -5230907.0          5226329.0   
2   2002  6.285683e+09          -6407331.0          6400746.0   
3   2003  6.366908e+09          -5928647.0          5925313.0   
4   2004  6.448284e+09          -5694224.0          5698402.0   
5   2005  6.529978e+09          -6497101.0          6504742.0   
6   2006  6.612296e+09          -8222083.0          8219823.0   
7   2007  6.694976e+09          -8515480.0          8514608.0   
8   2008  6.778716e+09          -7877773.0          7884360.0   
9   2009  6.862780e+09          -6594817.0          6602539.0   
10  2010  6.946900e+09          -5394260.0          5392038.0   
11  2011  7.030814e+09          -6555565.0          6521010.0   
12  

In [22]:
print("global_data   : ", global_data.columns.tolist())

global_data.to_csv ( os.path.join ( data_file_path, "Master_Global_Data_2000_2019.csv" ), index=False  )


global_data   :  ['year', 'population', 'migration_outflows', 'migration_inflows', 'migration_flow_percent', 'migration_net', 'migration_net_percent']
